In [4]:
from robobrowser import RoboBrowser
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

def get_number(s):
    try:
        return float(s.replace(",","."))
    except ValueError:
        return None

#Open URL and create RoboBrowser Instance
browser = RoboBrowser(parser='html.parser', history=False)
dates = pd.date_range('2016-01-01', periods=105, freq='W').format('YYYY-MM-dd')[1:] # weekly dates (always sunday)
print(dates)
results = pd.DataFrame()

for d in dates:
    
    url='https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/' + d + '/DE'
    print(url)
    browser.open(url)  
    soup = bs(str(browser.parsed), 'html.parser') # get html
    germanData = soup.find(id="tab_de")
    weekData = germanData.find("table", class_="hours") # select german hourly table (last on tob DE/AT)
    #print(soup.prettify())
    dates = pd.date_range(pd.to_datetime(d) - pd.to_timedelta(6, 'D'), periods=7, freq="D") # create dates for week
    hours = ["'00 - 01", "'01 - 02", "'02 - 03", "'03 - 04", "'04 - 05", "'05 - 06", 
             "'06 - 07", "'07 - 08", "'08 - 09", "'09 - 10", "'10 - 11", "'11 - 12", 
             "'12 - 13", "'13 - 14", "'14 - 15", "'15 - 16", "'16 - 17", "'17 - 18", 
             "'18 - 19", "'19 - 20", "'20 - 21", "'21 - 22", "'22 - 23", "'23 - 24"] # table column names; "'" at beginning for opening in excel
    priceRows = weekData.findAll("tr", class_="no-border") # get rows with €/Mwh
    if d == "2016-10-30": # two rows for 2-3 period for this week, only keep first
        del priceRows[3]
    prices = np.zeros((24, 7))
    for i in range(24):
        prices[i] = [get_number(p.get_text()) for p in priceRows[i].findAll("td")[2:]]
    prices = prices.transpose()
    df = pd.DataFrame(prices)
    df.index = dates
    df.columns = hours
    # cut off last dates of 2015
    if d == "2016-01-03":
        df = df[4:]
    #print(df)
    if results.empty:
        results = df
    else:
        results = results.append(df)
print(results)
results.to_csv("day_ahead_auction.csv", sep=";", decimal=",")


['2016-01-03', '2016-01-10', '2016-01-17', '2016-01-24', '2016-01-31', '2016-02-07', '2016-02-14', '2016-02-21', '2016-02-28', '2016-03-06', '2016-03-13', '2016-03-20', '2016-03-27', '2016-04-03', '2016-04-10', '2016-04-17', '2016-04-24', '2016-05-01', '2016-05-08', '2016-05-15', '2016-05-22', '2016-05-29', '2016-06-05', '2016-06-12', '2016-06-19', '2016-06-26', '2016-07-03', '2016-07-10', '2016-07-17', '2016-07-24', '2016-07-31', '2016-08-07', '2016-08-14', '2016-08-21', '2016-08-28', '2016-09-04', '2016-09-11', '2016-09-18', '2016-09-25', '2016-10-02', '2016-10-09', '2016-10-16', '2016-10-23', '2016-10-30', '2016-11-06', '2016-11-13', '2016-11-20', '2016-11-27', '2016-12-04', '2016-12-11', '2016-12-18', '2016-12-25', '2017-01-01', '2017-01-08', '2017-01-15', '2017-01-22', '2017-01-29', '2017-02-05', '2017-02-12', '2017-02-19', '2017-02-26', '2017-03-05', '2017-03-12', '2017-03-19', '2017-03-26', '2017-04-02', '2017-04-09', '2017-04-16', '2017-04-23', '2017-04-30', '2017-05-07', '2017

https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-07-23/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-07-30/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-08-06/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-08-13/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-08-20/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-08-27/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-09-03/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-09-10/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-09-17/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-09-24/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-10-01/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2017-10-08/DE
http